# Using the Dataloader

In [4]:
import numpy as np
import pandas as pd
import copy
import torch
import sys
import os
from pgmpy.models import BayesianModel
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.estimators import BayesianEstimator
from pgmpy.inference import VariableElimination
from torch.utils.data import Dataset, DataLoader
sys.path.append("./..")
import classes.powerdf as pdf
import matplotlib.pyplot as plt
from classes.data_loader import smarthome_dataset

In [13]:
cur_path = os.path.abspath(os.getcwd())
dataset_path = os.path.join(cur_path,"cleaned_HomeC.csv")
print(dataset_path)
df = pd.read_csv(dataset_path, low_memory = False)
dataset = smarthome_dataset("cleaned_HomeC.csv" , dayahead = True)
print("dataset length", len(dataset))

/Users/meezy/Documents/Carnegie/10-708_F23/GRIDPredict/notebooks/cleaned_HomeC.csv
dataset length 503910


In [14]:
# create training and testing dataset
# fill missing values in
df.fillna(value = 0)
train_df = df.iloc[:3000]
test_df = df.iloc[4000:4400]
df = df.iloc[:]

traindata_loader = DataLoader(dataset, batch_size=10, shuffle=True,
                              num_workers=0)
testdata_loader = DataLoader(dataset, batch_size=10, shuffle=True,
                             num_workers=0)

"""
open = True
while (open):
    try:
        inputs, label = next(iter(traindata_loader))  
        print(inputs.shape)
        print(label.shape)
        #print("input {} : label {}".format(inputs,label))
        open = True
    except:
        open = False
"""

'\nopen = True\nwhile (open):\n    try:\n        inputs, label = next(iter(traindata_loader))  \n        print(inputs.shape)\n        print(label.shape)\n        #print("input {} : label {}".format(inputs,label))\n        open = True\n    except:\n        open = False\n'

The data loader returns input features corresponding to the weather features and the label corresponds to the label.

If dayahead is True (predicting next hour's solar generation based on previous 24 hourly weather features)...

\begin{align}
\text{Input Size}: & (\text{batch size } \times \text{ weather features } \times \text{ past 24 hour data points})\\
\text{Labels}: & (\text{batch size})
\end{align}

If dayahead is False (predicting hours solar generation based on current weather features)

\begin{align}
\text{Input Size}: & (\text{batch size } \times \text{ weather features })\\
\text{Labels}: & (\text{batch size})
\end{align}

In [15]:
# Using MLP
from networks.mlp_baseline import *
mlp_model = MLP_baseline(24,9)
optimizer = torch.optim.Adam(mlp_model.parameters(), lr=0.001)
loss_fn = kl_sum_loss

print("Beginning Training...")
num_epochs = 1
for n in range(num_epochs):
    total_loss = 0
    for batch_idx, (x, y) in enumerate(traindata_loader):
        print("batch {}".format(batch_idx))
        x, y = next(iter(traindata_loader))
        y_pred = mlp_model(x)
        loss = loss_fn(y_pred, y)
        total_loss = loss + total_loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print("epoch loss:", loss.item())

Beginning Training...
batch 0


RuntimeError: mat1 and mat2 shapes cannot be multiplied (10x216 and 576x256)

In [ ]:
# evaluation
for batch_idx, (x, y) in enumerate(traindata_loader):
    print("batch {}".format(batch_idx))
    x, y = next(iter(train_dataloader))
    y_pred = mlp_model(x)
    loss = loss_fn(y_pred, y)
    total_loss = loss + total_loss
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
print("epoch loss:", loss.item())